In [ ]:
import tweepy
import json
import configparser
import logging
from helper_functions import *

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')

access_token = config['twitterAuth']['access_token']
access_token_secret = config['twitterAuth']['access_token_secret']
consumer_key = config['twitterAuth']['consumer_key']
consumer_secret = config['twitterAuth']['consumer_secret']


In [ ]:
# define stream listener class
class TwitterStreamListener(tweepy.Stream):
    def __init__(self, api=None):
        super(tweepy.Stream, self).__init__()
        # super(json.JSONEncoder, self).__init__()
        self.consumer_key = consumer_key
        self.consumer_secret = consumer_secret
        self.access_token = access_token
        self.access_token_secret = access_token_secret
        self.api = api

        self.num_tweets = 0
        # self.file = open('tweets.txt', 'w')
        self.tweet_list = []

    def toJson(self):
        return json.dumps(self, default=lambda o: o.__dict__)

    def on_status(self, status):  # returns JSON
        #print(status[0]._json)

        #tweet = status._json
        #print(status.id)
        # Retweet count
        # retweet_count = status['retweet_count']
        # status = status.toJson()
        # status = json.loads(status)
        # status = str(status)

        while self.num_tweets < 450:  # max stream rate is for the twitter API Client
            # for tweet in status:
            #     tweet = tweet._json
            #     print("printing twetets")
            #     print(tweet)
            try:
               # self.filter()
                with open('tweets.json', 'a') as f:
                    # write results to JSON file
                    print("Returning JSON-ish results as string")
                    json.dump(status._json, f)
                    #f.write(tweet)
                    #return True

            except TypeError as e:
                logging('Error: ', e)
                print('{e}: Please convert Stream object')
                continue
            # TODO: except user exit or stream disconnect
            else:
                with open('tweets.json', 'r') as file:
                    # create Python list object from JSON
                    tweets_json = file.read().split("\n")

                    for tweet in tweets_json:
                        print("Deserializing tweets")
                        tweet_obj = json.loads(tweet)

                        # Tweet ID
                        tweet_obj['tweet_id'] = tweet_obj['id']
                        # User ID
                        tweet_obj['user_id'] = tweet_obj['user']['id']
                        # Username
                        tweet_obj['username'] = tweet_obj['user']['name']
                        # creation date
                        tweet_obj['create_time'] = tweet_obj['user']['creation_date']
                        # Language
                        lang = status['lang']

                        # Tweet
                        if status.truncated == True:
                            tweet = tweet_obj['extended_tweet']['full_text']
                            hashtags = tweet_obj['extended_tweet']['entities']['hashtags']
                        else:
                            tweet = status.text
                            hashtags = status.entities['hashtags']

                        # Read hashtags using helper function
                        print("Reading hashtags")
                        hashtags = read_hashtags(hashtags)

                        # add info to pop_news dict
                        # If tweet is not a retweet and tweet is in English
                        if not hasattr(status, "retweeted_status") and lang == "en":
                            print("Adding tweets to list")
                            self.tweet_list.append(tweet_obj)

                        #self.tweet_list.append(status)
                            self.num_tweets += 1
            finally:
                self.disconnect()
                return self.tweet_list

            # flatten data to dataframe
            # tweets = pd.json_normalize(self.tweet_list, record_path=['articles'])
        #self.tweets_df = pd.DataFrame(self.tweet_list, columns=[
                #"tweet_id", "publishedAt", "userID", "text", "location"])

    def on_data(self, data):
        # payload = {}
        data = json.loads(data)
        print(data)

    # Extract hashtags
    def read_hashtags(self, tag_list):
        hashtags = []

        for tag in tag_list:
            hashtags.append(tag['text'])

        return hashtags

    def clean_tweets(self):

        with open("tweets.json", "w") as f:
            # write tweets to json file
            json.dump(tweet, f)

        with open("tweets.json", "r") as file:
            # create python object from json
            tweets_json = file.read().split("\n")

            for tweet in tweets_json:
                tweet_obj = json.loads(tweet)

                #flatten nested fields
                if 'quoted_status' in tweet_obj:
                    tweet_obj['quote_tweet'] = tweet_obj['quoted_status']['extended_tweet']['full_text']
                if 'user' in tweet_obj:
                    tweet_obj['location'] = tweet_obj['user']['location']
                # if 'created_at' in tweet_obj:
                #     tweet_obj['created_at'] = pd.to_datetime(tweet)

    def on_error(self, status_code):
        if status_code == 420:
            return False  # false disconnects the stream


In [ ]:

auth = t.tweepy_auth()
# instantiate Tweet Stream Listener
listener = TwitterStreamListener()
# authenticate stream
# tweet_mode="extended")
tweet_stream = tweepy.Stream(auth, listener, access_token, access_token_secret)
#listener.on_status(tweet_stream)
listener.on_data(tweet_stream)

# print cleaned tweets df

# print(news.all_news_df)
